In [32]:
from random import randint
import datetime
import numpy as np

In [33]:
# UnixTime
t2010 = 1262282400
DEVICE = "cuda:0"

In [91]:
dataset_in = []
dataset_out = []
for _ in range(100000):
    v_p = randint(400,10000)
    x0 = randint(-18000,18000)
    y0 = randint(-9000,9000)
    z0 = randint(-5000,0)
    t0 = randint(1, 648023579)
    t0 = t0 - (t0//1000)*1000
    dataset_out.append([x0,y0,z0,t0])
    ins = []
    for i in range(3):
        x = x0 + randint(-100,100)
        y = y0 + randint(-100,100)
        z = z0 + randint(-200,2000)
        t_p = t0 + (1 / v_p) * (((x - x0)**2 + (y - y0)**2 + (z - z0)**2)**0.5) - (t0//1000)*1000
        ins.append(x)
        ins.append(y)
        ins.append(z)
        ins.append(t_p)
    dataset_in.append(ins)

print(dataset_in[0], dataset_out[0])
np.save('in.npy',np.array(dataset_in))
np.save('out.npy',np.array(dataset_out))

[-8720, 5382, -4256, 552.0551640843423, -8585, 5364, -3329, 552.1540538940317, -8722, 5490, -4369, 552.0432649015188] [-8664, 5437, -4766, 552]


In [92]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from sklearn.preprocessing import StandardScaler

class BostonDataset(torch.utils.data.Dataset):
  '''
  Prepare the Boston dataset for regression
  '''

  def __init__(self, X, y, fit_scale=True):
    if not torch.is_tensor(X) and not torch.is_tensor(y):
      # Apply scaling if necessary
      if fit_scale:
          self.scalerx = StandardScaler().fit(X)
          self.scalery = StandardScaler().fit(y)
      X = self.scalerx.transform(X)
      y = self.scalery.transform(y)
      self.X = torch.from_numpy(X)
      self.y = torch.from_numpy(y)

  def __len__(self):
      return len(self.X)

  def __getitem__(self, i):
      return self.X[i], self.y[i]

  def get_scaler(self):
      return self.scalerx, self.scalery


class MLP(nn.Module):
  '''
    Multilayer Perceptron for regression.
  '''
  def __init__(self):
    super().__init__()
    self.layers = nn.Sequential(
      nn.Linear(12, 64),
      nn.LeakyReLU(),
      nn.Linear(64, 64),
      nn.LeakyReLU(),
      nn.Linear(64, 4)
    )


  def forward(self, x):
    x = self.layers(x)
    return x


if __name__ == '__main__':

  # Set fixed random number seed
  torch.manual_seed(42)

  I = []
  O = []
  for i in dataset_in:
    I.append(np.array(i))
  for i in dataset_out:
    O.append(np.array(i))
  dataset = BostonDataset(np.array(I), np.array(O))


  trainloader = torch.utils.data.DataLoader(dataset, batch_size=10, shuffle=True)

  # Initialize the MLP
  mlp = MLP().to(DEVICE)

  # Define the loss function and optimizer
  loss_function = nn.MSELoss()
  optimizer = torch.optim.Adam(mlp.parameters(), lr=0.001)

  # Run the training loop
  for epoch in range(0, 5): # 5 epochs at maximum

    # Print epoch
    print(f'Starting epoch {epoch+1}')

    # Set current loss value
    current_loss = 0.0

    # Iterate over the DataLoader for training data
    for i, data in enumerate(trainloader, 0):

      # Get and prepare inputs
      inputs, targets = data
      inputs, targets = inputs.float(), targets.float()
      targets = targets.reshape((targets.shape[0], -1))

      # Zero the gradients
      optimizer.zero_grad()

      # Perform forward pass
      outputs = mlp(inputs.to(DEVICE))

      # Compute loss
      loss = loss_function(outputs, targets.to(DEVICE))

      # Perform backward pass
      loss.backward()

      # Perform optimization
      optimizer.step()

      # Print statistics
      current_loss += loss.item()
      if i % 10 == 0:
          print('Loss after mini-batch %5d: %.3f' %
                (i + 1, current_loss))
          current_loss = 0.0

  # Process is complete.
  print('Training process has finished.')

Выходные данные были обрезаны до нескольких последних строк (5000).
Loss after mini-batch    51: 2.994
Loss after mini-batch    61: 1.440
Loss after mini-batch    71: 0.921
Loss after mini-batch    81: 0.456
Loss after mini-batch    91: 0.419
Loss after mini-batch   101: 0.296
Loss after mini-batch   111: 0.278
Loss after mini-batch   121: 0.222
Loss after mini-batch   131: 0.238
Loss after mini-batch   141: 0.196
Loss after mini-batch   151: 0.244
Loss after mini-batch   161: 0.199
Loss after mini-batch   171: 0.201
Loss after mini-batch   181: 0.241
Loss after mini-batch   191: 0.228
Loss after mini-batch   201: 0.185
Loss after mini-batch   211: 0.197
Loss after mini-batch   221: 0.235
Loss after mini-batch   231: 0.191
Loss after mini-batch   241: 0.201
Loss after mini-batch   251: 0.192
Loss after mini-batch   261: 0.192
Loss after mini-batch   271: 0.189
Loss after mini-batch   281: 0.193
Loss after mini-batch   291: 0.191
Loss after mini-batch   301: 0.241
Loss after mini-batch 

In [93]:
data = torch.Tensor(np.array([13862, 8987, -174, 86.35223236093526, 13826, 8947, -1655, 86.08302615857285, 13807, 8848, -1233, 86.15929224942208]).reshape(1, -1))
xscaler, yscaler = dataset.get_scaler()
output = mlp(torch.Tensor(xscaler.transform(data)).to(DEVICE))
print(yscaler.inverse_transform(output.tolist()))


[[13929.72117531  8906.46604459 -1784.24737166    75.24480098]]


In [ ]:
[13784, 8899, -2107, 86]


1 [[14117.08477714  9026.49137562 -2026.53661859    84.4427884 ]]
2 [[13816.32684224  8796.13465645 -1856.22839095    77.51776314]]
3 [[14138.54167975  9072.3426838  -1893.57716038    85.19811237]]
